<a href="https://colab.research.google.com/github/PhilWa/Deep_learning_explorations/blob/master/Generator_ms2fp_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting MACCs from MS2 data




To let colab run longer
- https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

To perform hyperparameter search
- https://maxpumperla.com/hyperas/



## Defining experimental variables

In [0]:
experiment_name = "something_light_ms2fp"
data_dir = "/content/drive/My Drive/MS2 predictions/data/"
output_dir = "/content/drive/My Drive/MS2 predictions/"
save_model = False

The aim of this setup is to train a model that beats Sirius4.
Therefor different experiments will be performed. <br>

The logic will be: Iterate over model structures (currently it looks like the bigger the model the better it performs), each with different data generators.
Training will happen on 90% of the data. 10% will be validation data to ensure there is no overfitting. After each run (100 epochs) or if validation loss doesnt descrease in 5 consecutive runs.

Then we compare the prediction to the output from Sirius4 from the CASMI challenges.

To this end the hamming distance between:
  - ground truth and Sirius4 fingerprints = sirius_Ham
  - ground truht and current model finger print = swiss_Ham

> if sirius_Ham > swiss_Ham <br>
> print("Happy!")





In [0]:
## add a .txt to add more meta info 

## Setup environment

In [0]:
 from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-beta1
#import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import io
import numpy as np
import datetime, os
from collections import Counter
import matplotlib.pyplot as plt
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Helper functions

- `smart_train_split` Function to split data in a way that there are only never before seen finger prints in the test data set
- `compute_sample_weights` Function to weigh samples by the number of their occurance in the training data set
- `compute_class_weights` Function to weigh classes by the number of their occurance in the training data set
- `binarize_pred`
- `evaluate_prediction`
- `generate_ms2fp_data`

In [0]:
def smart_train_split(ms2_infos, finger_prints,random_seed, train_percentage, validation_fraction):
    np.random.seed(random_seed)
    samples_ids = finger_prints['duplicated_id'].values
    unique_sample_ids = np.unique(samples_ids)
    max_for_split = int(round(len(unique_sample_ids) * train_percentage))
    print(max_for_split,len(unique_sample_ids), train_percentage)
    sample_selection = np.random.choice(unique_sample_ids, size=max_for_split)
    selection_bool = np.isin(samples_ids, sample_selection)

    print("Train/Test Split")
    test_y = finger_prints[selection_bool]
    train_temp_y = finger_prints[np.invert(selection_bool)]

    test_x = ms2_infos[selection_bool]
    train_temp_x = ms2_infos[np.invert(selection_bool)]
    print("Train/Test Split - Done")

    print("Validation/Train Split")

    validation_sample_ids = samples_ids[np.invert(selection_bool)]
    unique_validation_ids = np.unique(validation_sample_ids)
    max_for_validation_split = int(round(len(unique_validation_ids)*validation_fraction))
    validation_selection = np.random.choice(unique_validation_ids, size = max_for_validation_split)
    validation_bool = np.isin(validation_sample_ids, validation_selection)

    validation_y = train_temp_y[validation_bool]
    train_y = train_temp_y[np.invert(validation_bool)]

    validation_x = train_temp_x[validation_bool]
    train_x = train_temp_x[np.invert(validation_bool)]

    print("Validation/Train Split - Done")
    print("______________________________")
    print(train_y.shape, "train_y")
    print(train_x.shape, "train_x")
    print(test_y.shape, "test_y")
    print(test_x.shape, "test_x")
    print(validation_y.shape, "validation_y")
    print(validation_x.shape, "validation_x")
    
    return train_x, test_x, validation_x, train_y, test_y, validation_y


def compute_sample_weights(y_,plot_hist=False):
    duplicated_ids=y_["duplicated_id"]
    #Compute sample_weight_vector
    from collections import Counter
    cnt = Counter()
    for occurance in duplicated_ids:
        cnt[occurance] += 1
    final = Counter(cnt)
    
    freq_collection = pd.DataFrame.from_dict(final, orient="index").reset_index()
    freq_collection = freq_collection.rename(columns={"index":"Duplicated_id",0:"Frequency"})
    freq_collection["Duplicated_id"] = freq_collection["Duplicated_id"].apply(str)

    duplicated_ids = pd.DataFrame(duplicated_ids)
    y_ids = duplicated_ids.rename(columns={"duplicated_id":"Duplicated_id"})
    y_ids["Duplicated_id"] = y_ids["Duplicated_id"].apply(str)

    y_ids = y_ids.merge(freq_collection,on="Duplicated_id", suffixes=('_y_ids','_freq_ids'))
    sample_weight_vector = y_ids["Frequency"]
    if plot_hist:
      plt.hist(sample_weight_vector)
    
    return sample_weight_vector

def compute_class_weights(y_, plot_hist=False):
    y_clean = y_.drop("duplicated_id",axis=1)
    equal_one = (y_clean==1).sum()
    class_weight_vector=(max(equal_one)/equal_one)
    inf_bool = np.isinf(class_weight_vector)
    class_weight_vector[inf_bool]=0
    if plot_hist:
      plt.hist(class_weight_vector, bins=40)
    
    return class_weight_vector



from progressbar import *               # just a simple progress bar

widgets = ['Progess: ', Percentage(), ' ', Bar(marker='#',left='[',right=']'),
           ' ', ETA(), ' ', FileTransferSpeed()] #see docs for other options

# pbar = ProgressBar(widgets=widgets, maxval=500)
# pbar.start()

# for i in range(100,500+1,50):
#     # here do something long at each iteration
#     pbar.update(i) #this adds a little symbol at each iteration
# pbar.finish()



    
def binarize_pred(pred_fp, cutoff_value = 0.5):
    pred_fp=pd.DataFrame(pred_fp)
    pred_fp[pred_fp<cutoff_value]=0
    pred_fp[pred_fp>=cutoff_value]=1
    return pred_fp


def evaluate_prediction(test_Y_clean, predicted_Y_df, return_hamming_distance = False, save_output = False):
    from scipy.spatial import distance
    hamming_pred = []
    hamming_rand = []
    hamming_superrand = []
    pbar = ProgressBar(widgets=widgets, maxval=test_Y_clean.shape[0])
    pbar.start()

    for index, fingerprint in test_Y_clean.iterrows():
      predicted_fingerprint = predicted_Y_df.iloc[index]
      random_int = np.random.randint(0,test_Y_clean.shape[0])
      random_predicted_fingerprint = predicted_Y_df.iloc[random_int]
      hamming_pred.append(distance.hamming(fingerprint,predicted_fingerprint))
      hamming_rand.append(distance.hamming(fingerprint,random_predicted_fingerprint))
      hamming_superrand.append(distance.hamming(np.random.randint(0,2, 152),np.random.randint(0,2, 152)))
      pbar.update(index)

    pbar.finish()

    median_distance = np.median(hamming_rand)-np.median(hamming_pred)
    plt.hist(hamming_pred, alpha=0.4,color='B', label='Predicted Fingerprint')
    plt.hist(hamming_rand, alpha=0.4,color='R', label='Random Fingerprint')
    plt.hist(hamming_superrand, alpha=0.4,color='G', label='Random binary vector')
    plt.title('Hamming distance ' + experiment_name + " avg:s "+str(np.round(median_distance, 3)))
    plt.axvline(np.median(hamming_pred), color='B', linestyle=':', linewidth=1)
    plt.axvline(np.median(hamming_rand), color='R', linestyle='dashed', linewidth=1)
    plt.axvline(np.median(hamming_superrand), color='G', linestyle='dashed', linewidth=1)
    plt.legend(loc='upper right')
    if save_output: 
      filepath=data_dir + dt_string + "_" + experiment_name + "hamming_distribution" + ".png"
      plt.savefig(filepath)
    plt.show()
    
    
    print(np.mean(median_distance))
    if return_hamming_distance:
      return hamming_pred

def generate_ms2fp_data(ms2_info,
                        finger_prints,
                        batch_size = 100,
                        flip= True,
                        drop_peaks = True,
                        output_shape = "Conv1D" ,
                        generator_mode = "Train"):
    i = 0
    ms2_info = ms2_info.values
    finger_prints = finger_prints.values
    while True:
        ms2_info_out = np.zeros((batch_size,ms2_info.shape[1]))
        finger_prints_out = np.zeros((batch_size,finger_prints.shape[1]))
        for b in range(batch_size):
            if i == ms2_info.shape[0]:
                i = 0
                
                shuffle_numbers = np.random.randint(0,ms2_info.shape[0],ms2_info.shape[0])
                ms2_info = ms2_info[shuffle_numbers,:]
                finger_prints = finger_prints[shuffle_numbers,:] 
            
            if generator_mode == "Train":
                meta_info = ms2_info[i,:3]
                noised_ms = ms2_info[i,3:] * np.random.normal(1,0.1,1)
                
                if drop_peaks:
                    ms_info = np.concatenate([meta_info, noised_ms])
                    non_zeros = np.asarray(np.nonzero(ms_info[3:]))[0]
                    zero_id = np.random.randint(3, len(non_zeros),int(len(non_zeros)*0.1)) 
                    ms_info_dropped = ms_info[zero_id] = 0
                    ms2_info_out[b,3:] = ms_info_dropped
                else:
                    ms2_info_out[b,:] = np.concatenate([meta_info, noised_ms])

                finger_prints_out[b,:] = finger_prints[i,:]
            if generator_mode == "Validate":
                
                finger_prints_out[b,:] = finger_prints[i,:]
                ms2_info_out[b,:] = ms2_info[i,:]
            
            i += 1
        
        if flip & (generator_mode == "Train"):
            ms2_info_out[1::2, :] = ms2_info_out[1::2, ::-1]
        
        if output_shape == "Conv1D":
            ms2_info_out =  np.reshape(ms2_info_out, (ms2_info_out.shape[0],ms2_info_out.shape[1],1))

        yield (ms2_info_out, finger_prints_out) 
        

## Load and prepare data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
y_in=pd.read_pickle(data_dir + "y_in.pkl")
x_in=pd.read_pickle(data_dir + "x_in.pkl")

In [0]:
import re
y_colnames=y_in.columns.values.tolist()
y_colnames_clean=[re.sub("MACC", "", x) for x in y_colnames]

y_colnames_clean = y_colnames_clean[:-1]
y_colnames_clean = [int(i) for i in y_colnames_clean] 

In [0]:
empty_y=np.zeros((y_in.shape[0],166))

In [0]:
empty_y[:,y_colnames_clean]=

In [0]:
casmi_groundtruth_y = pd.read_csv(data_dir + "calculated_maccs_casmi.csv") 

In [0]:
casmi_groundtruth_y = casmi_groundtruth_y.drop("Unnamed: 0", axis=1)

In [0]:
casmi_groundtruth_y

Double check that the way the weight_vectors are designed on the splitted data


In [0]:
train_X, test_X, validation_X, train_Y, test_Y, validation_Y  =  smart_train_split(x_in ,y_in , 41, 0.2, 0.1)

In [0]:
del(y_in)
del(x_in)

In [0]:
sample_weights = compute_sample_weights(train_Y,plot_hist=False)
validation_sample_weights =  compute_sample_weights(validation_Y,plot_hist=False)
class_weights = compute_class_weights(train_Y,plot_hist=False)

In [0]:
test_Y = test_Y.drop("duplicated_id",axis=1)
train_Y = train_Y.drop("duplicated_id",axis=1)
validation_Y = validation_Y.drop("duplicated_id", axis=1)

## Feature curation 



## Setup 1Dconv model

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(input_shape=(4444,1), kernel_size=32,filters=512, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(152, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


model.summary()

# checkpoint to save best model
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y")

filepath= data_dir + dt_string + "_" + experiment_name + "model_visualization" + ".png"
tf.keras.utils.plot_model(model, to_file=filepath, show_shapes =True)


### Set up callback list

Generate callback_lists to:
- Save best model
- Reduce learning rate upon val_loss plateau
- 

In [0]:

filepath=data_dir + dt_string + experiment_name + ".hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Adjust learning rate upon plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=0.001)

callbacks_list = [reduce_lr]

## Train model

In [0]:
train_generator = generate_ms2fp_data(train_X,train_Y, 100,drop_peaks=False, generator_mode = "Train")
validation_generator = generate_ms2fp_data(validation_X,validation_Y, 100, generator_mode = "Validation")

In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=1000,
                              validation_data = validation_generator, 
                              validation_steps = 10,
                              epochs=3,
                              verbose=1, callbacks=callbacks_list)


"""
fit(x=train_1D_X,
            y=np.abs(train_Y),
            validation_data = (validation_1D_X, np.abs(validation_Y), validation_sample_weights),
            shuffle=True,
            epochs=5,
            batch_size=100,
            class_weight=class_weights,
            sample_weight=sample_weights,
            callbacks = callbacks_list)"""

## Investigate output of model


In [0]:
last_trained_model = model

In [0]:
test_1D_X = np.abs(test_X).values.reshape(test_X.shape[0],test_X.shape[1],1)
#model.evaluate(x=np.abs(test_1D_X), y=np.abs(test_Y))

In [0]:
#model_dir = data_dir + dt_string + experiment_name + ".hdf5"
#model = tf.keras.models.load_model(model_dir)

In [0]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss ' + experiment_name)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
filepath=data_dir + dt_string + "_" + experiment_name + "loss_plot" + ".png"
plt.savefig(filepath)
plt.show()


In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy ' + experiment_name)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
filepath=data_dir + dt_string + "_" + experiment_name + "accuracy_plot" + ".png"
plt.savefig(filepath)
plt.show()

In [0]:
predicted_Y = model.predict(x=np.abs(test_1D_X))

In [0]:
predicted_cut_Y = binarize_pred(predicted_Y,cutoff_value=.5)

In [0]:
tested_Y = test_Y.reset_index().drop(["index"], axis=1)

In [0]:
evaluate_prediction(tested_Y, predicted_cut_Y, save_output= True)

In [0]:
import seaborn as sns

difference_matrix =(predicted_cut_Y.astype("int").values==tested_Y.values).astype("int")
range_plot = range(difference_matrix.shape[0])

difference_count = difference_matrix.shape[1]-np.sum(difference_matrix, axis=1)
data = {"Count":difference_count, "Sample_id":range(difference_matrix.shape[0])}

fig, ax = plt.subplots(1,5)
sns.heatmap(tested_Y,cbar=False, xticklabels=False, yticklabels=False, ax=ax[0]).set_title('Y_test')
sns.heatmap(predicted_cut_Y,cbar=False, xticklabels=False, yticklabels=False, ax=ax[1]).set_title('Y_pred')
sns.heatmap(difference_matrix, xticklabels=False, yticklabels=False, ax=ax[2]).set_title('1-Diff')

fig.tight_layout()
fig.show()




In [0]:
test_Y=test_Y.reset_index(drop=True)
rowsum = np.sum(test_Y, axis = 1)
colsum = np.sum(test_Y, axis = 0)
plt.stem(rowsum, use_line_collection=True, markerfmt="")

In [0]:
plt.stem(colsum, use_line_collection=True, markerfmt="")

## Save output


### Save predictions as csv


In [0]:
filepath=data_dir + dt_string + experiment_name + "y_pred" + ".csv"
predicted_Y.to_csv(filepath)
filepath=data_dir + dt_string + experiment_name + "y_test" + ".csv"
test_Y.to_csv(filepath)

Save 